In [6]:
# from selenium import webdriver

# pip install pandas in command line
import pandas as pd

# import packages to set wait time for page to load before running Selenium code
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# import time 
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver import ActionChains

# packages to download images
import urllib
import urllib.request

# regex
import re

# sorting
import string

In [ ]:
letters = string.ascii_uppercase
df = pd.DataFrame(columns = ['name', 'uni', 'title', 'department'])

username = input("Username:")
password = input("Password:")

driver = webdriver.Chrome()

for i in letters:
    driver.get("https://directory.columbia.edu/people/browse/students;jsessionid=AE18155A2A30B9D6D4AEE0653247E6A4?filter.initialLetter=" + str(i))

    try:
        driver.find_element_by_id("username").send_keys(username)
        driver.find_element_by_id("password").send_keys(password)
        driver.find_element_by_xpath('//*[@id="fm1"]/div[3]/input[4]').send_keys(Keys.RETURN)

    except:
        pass

    current_url = driver.current_url
    ending = "filter.initialLetter=" + str(i)
    print(ending)
    print(len(df.name))
    
    try:
        for i in range(0,150):
            new_ending = 'page=' + str(i)

            new_url = re.sub(ending, new_ending, current_url)
            #print(new_url)

            driver.get(new_url)

            for i in range(2,42):
                s = []
                try:
                    name = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[1]/a')
                    s.append(name.text)
                
                    try:
                        uni = name.get_attribute('href')
                        s.append(uni)
                        
                        try:
                            department = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[2]/div[2]')
                            s.append(department.text)

                            try:
                                title = driver.find_element_by_xpath('/html/body/div/a/form/table/tbody/tr[5]/td[2]/div[1]/table/tbody/tr[' + str(i) + ']/td[2]/div[1]')
                                s.append(title.text)
                            
                            except:
                                s.append("None")
                        except:
                            pass
                    except:
                        pass
                except:
                    pass
                
                df.loc[len(df)] = s
    except:
        pass

driver.close()

In [104]:
# clean uni column
for i in range(0,len(df2['uni'])):
               df2['uni_2'][i] = re.search('(?<=\=).+', df2['uni'][i]).group(0)


In [113]:
# clean title column
df2['title_2'] = df2['title'].str[12:]

In [118]:
# clean department column
df2['department_2'] = df2['department'].str[7:]

In [120]:
# drop uni, title, department
df2.drop(['department', 'uni', 'title'], inplace = True, axis =1)

In [122]:
# rename cols
df2.columns = ['name', 'uni', 'title', 'department']

In [124]:
# save full directory
df2.to_csv('CU_directory.csv')

In [125]:
# only CBS
df_CBS = df2[df2['department'] == "Student, GRADUATE SCHOOL OF BUSINESS"]

In [127]:
# reset index
df_CBS.reset_index(inplace= True)
df_CBS.drop('index', inplace = True, axis =1)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [129]:
# drop department and title
df_CBS.drop(['department', 'title'], inplace = True, axis =1)

In [ ]:
# save CBS directory
df.to_csv('CBS_directory.csv')

In [26]:
df2 = pd.read_csv('./CBS_directory_rough.csv')

In [27]:
df2.drop("Unnamed: 0", inplace = True, axis = 1)

In [106]:
df2[df2['name'] == "Gelly, Bennett Butler Vaughan"]

,name,uni,title,department,uni_2
11109,"Gelly, Bennett Butler Vaughan",https://directory.columbia.edu/people/uni?code...,Department:\nBusiness,"Title:\nStudent, GRADUATE SCHOOL OF BUSINESS",bbg2123


In [43]:
lambda x: re.search(r'(?<=\=).+', x),s

(<function __main__.<lambda>(x)>, ['=abcdef'])

In [42]:
s = ['=abcdef']

In [128]:
df_CBS

,name,uni,title,department
0,"Abad Vime, Jose Miguel",ja3268,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
1,"Abarcar, Jc",jfa2127,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
2,"Abburi, Praneeth",ja3438,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
3,"Abdelhak, Serge",sa3699,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
4,"Abi Diwan, David",da2865,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
5,"Abney, Isa Ishmael",iia2103,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
6,"Abraham, Alexander Hess",aha2156,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
7,"Abrol, Aditi Ajay",aaa2352,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
8,"Abu Hawili, Anas",aa4385,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
9,"Acenov, Grigor",ga2492,Business,"Student, GRADUATE SCHOOL OF BUSINESS"
